# Import packages

In [1]:
import pandas as pd
import numpy as np
import torch
import pandas as pd
from pathlib import Path

from lightning import pytorch as pl
from lightning.pytorch.callbacks import ModelCheckpoint

from chemprop import data, featurizers, models, nn, uncertainty
from chemprop.models import save_model, load_model
from chemprop.cli.conf import NOW
from chemprop.data import build_dataloader
from chemprop.cli.predict import find_models

%load_ext autoreload
%autoreload 2

# Training

# Change data inputs here

In [2]:
chemprop_dir = Path.cwd().parent
input_path = (
    chemprop_dir / "tests" / "data" / "regression" / "mol" / "mol.csv"
)  # path to your data .csv file
num_workers = 0  # number of workers for dataloader. 0 means using main process for data loading
smiles_column = "smiles"  # name of the column containing SMILES strings
target_columns = ["lipo"]  # list of names of the columns containing targets

## Load data

In [3]:
df_input = pd.read_csv(input_path)
df_input

,smiles,lipo
0,Cn1c(CN2CCN(CC2)c3ccc(Cl)cc3)nc4ccccc14,3.54
1,COc1cc(OC)c(cc1NC(=O)CSCC(=O)O)S(=O)(=O)N2C(C)...,-1.18
2,COC(=O)[C@@H](N1CCc2sccc2C1)c3ccccc3Cl,3.69
3,OC[C@H](O)CN1C(=O)C(Cc2ccccc12)NC(=O)c3cc4cc(C...,3.37
4,Cc1cccc(C[C@H](NC(=O)c2cc(nn2C)C(C)(C)C)C(=O)N...,3.10
...,...,...
95,CC(C)N(CCCNC(=O)Nc1ccc(cc1)C(C)(C)C)C[C@H]2O[C...,2.20
96,CCN(CC)CCCCNc1ncc2CN(C(=O)N(Cc3cccc(NC(=O)C=C)...,2.04
97,CCSc1c(Cc2ccccc2C(F)(F)F)sc3N(CC(C)C)C(=O)N(C)...,4.49
98,COc1ccc(Cc2c(N)n[nH]c2N)cc1,0.20


# Get SMILES and targets

In [4]:
smis = df_input.loc[:, smiles_column].values
ys = df_input.loc[:, target_columns].values

In [5]:
smis[:5]  # show first 5 SMILES strings

array(['Cn1c(CN2CCN(CC2)c3ccc(Cl)cc3)nc4ccccc14',
       'COc1cc(OC)c(cc1NC(=O)CSCC(=O)O)S(=O)(=O)N2C(C)CCc3ccccc23',
       'COC(=O)[C@@H](N1CCc2sccc2C1)c3ccccc3Cl',
       'OC[C@H](O)CN1C(=O)C(Cc2ccccc12)NC(=O)c3cc4cc(Cl)sc4[nH]3',
       'Cc1cccc(C[C@H](NC(=O)c2cc(nn2C)C(C)(C)C)C(=O)NCC#N)c1'],
      dtype=object)

In [6]:
ys[:5]  # show first 5 targets

array([[ 3.54],
       [-1.18],
       [ 3.69],
       [ 3.37],
       [ 3.1 ]])

## Get molecule datapoints

In [7]:
all_data = [data.MoleculeDatapoint.from_smi(smi, y) for smi, y in zip(smis, ys)]

## Perform data splitting for training, validation, and testing

In [8]:
# available split types
list(data.SplitType.keys())

['CV_NO_VAL',
 'CV',
 'SCAFFOLD_BALANCED',
 'RANDOM_WITH_REPEATED_SMILES',
 'RANDOM',
 'KENNARD_STONE',
 'KMEANS']

In [9]:
mols = [d.mol for d in all_data]  # RDkit Mol objects are use for structure based splits
train_indices, val_indices, test_indices = data.make_split_indices(mols, "random", (0.8, 0.1, 0.1))
train_data, val_data, test_data = data.split_data_by_indices(
    all_data, train_indices, val_indices, test_indices
)

## Get MoleculeDataset

In [10]:
featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()

train_dset = data.MoleculeDataset(train_data, featurizer)
scaler = train_dset.normalize_targets()

val_dset = data.MoleculeDataset(val_data, featurizer)
val_dset.normalize_targets(scaler)

test_dset = data.MoleculeDataset(test_data, featurizer)

## Get DataLoader

In [11]:
train_loader = data.build_dataloader(train_dset, num_workers=num_workers)
val_loader = data.build_dataloader(val_dset, num_workers=num_workers, shuffle=False)
test_loader = data.build_dataloader(test_dset, num_workers=num_workers, shuffle=False)

# Change Message-Passing Neural Network (MPNN) inputs here

## Message Passing
A `Message passing` constructs molecular graphs using message passing to learn node-level hidden representations.

Options are `mp = nn.BondMessagePassing()` or `mp = nn.AtomMessagePassing()`

In [12]:
mp = nn.BondMessagePassing()

## Aggregation
An `Aggregation` is responsible for constructing a graph-level representation from the set of node-level representations after message passing.

Available options can be found in ` nn.agg.AggregationRegistry`, including
- `agg = nn.MeanAggregation()`
- `agg = nn.SumAggregation()`
- `agg = nn.NormAggregation()`

In [13]:
print(nn.agg.AggregationRegistry)

ClassRegistry {
    'mean': <class 'chemprop.nn.agg.MeanAggregation'>,
    'sum': <class 'chemprop.nn.agg.SumAggregation'>,
    'norm': <class 'chemprop.nn.agg.NormAggregation'>
}


In [14]:
agg = nn.MeanAggregation()

## Feed-Forward Network (FFN)

A `FFN` takes the aggregated representations and make target predictions.

Available options can be found in `nn.PredictorRegistry`.

For regression:
- `ffn = nn.RegressionFFN()`
- `ffn = nn.MveFFN()`
- `ffn = nn.EvidentialFFN()`

For classification:
- `ffn = nn.BinaryClassificationFFN()`
- `ffn = nn.BinaryDirichletFFN()`
- `ffn = nn.MulticlassClassificationFFN()`
- `ffn = nn.MulticlassDirichletFFN()`

For spectral:
- `ffn = nn.SpectralFFN()` # will be available in future version

In [15]:
print(nn.PredictorRegistry)

ClassRegistry {
    'regression': <class 'chemprop.nn.predictors.RegressionFFN'>,
    'regression-mve': <class 'chemprop.nn.predictors.MveFFN'>,
    'regression-evidential': <class 'chemprop.nn.predictors.EvidentialFFN'>,
    'classification': <class 'chemprop.nn.predictors.BinaryClassificationFFN'>,
    'classification-dirichlet': <class 'chemprop.nn.predictors.BinaryDirichletFFN'>,
    'multiclass': <class 'chemprop.nn.predictors.MulticlassClassificationFFN'>,
    'multiclass-dirichlet': <class 'chemprop.nn.predictors.MulticlassDirichletFFN'>,
    'spectral': <class 'chemprop.nn.predictors.SpectralFFN'>
}


In [16]:
output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)

In [17]:
# Change to other predictor if needed.
ffn = nn.RegressionFFN(output_transform=output_transform)

## Batch Norm
A `Batch Norm` normalizes the outputs of the aggregation by re-centering and re-scaling.

Whether to use batch norm

In [18]:
batch_norm = False

## Metrics
`Metrics` are the ways to evaluate the performance of model predictions.

Available options can be found in `metrics.MetricRegistry`, including

In [19]:
print(nn.metrics.MetricRegistry)

ClassRegistry {
    'mae': <class 'chemprop.nn.metrics.MAEMetric'>,
    'mse': <class 'chemprop.nn.metrics.MSEMetric'>,
    'rmse': <class 'chemprop.nn.metrics.RMSEMetric'>,
    'bounded-mae': <class 'chemprop.nn.metrics.BoundedMAEMetric'>,
    'bounded-mse': <class 'chemprop.nn.metrics.BoundedMSEMetric'>,
    'bounded-rmse': <class 'chemprop.nn.metrics.BoundedRMSEMetric'>,
    'r2': <class 'chemprop.nn.metrics.R2Metric'>,
    'roc': <class 'chemprop.nn.metrics.BinaryAUROCMetric'>,
    'prc': <class 'chemprop.nn.metrics.BinaryAUPRCMetric'>,
    'accuracy': <class 'chemprop.nn.metrics.BinaryAccuracyMetric'>,
    'f1': <class 'chemprop.nn.metrics.BinaryF1Metric'>,
    'bce': <class 'chemprop.nn.metrics.BCEMetric'>,
    'ce': <class 'chemprop.nn.metrics.CrossEntropyMetric'>,
    'binary-mcc': <class 'chemprop.nn.metrics.BinaryMCCMetric'>,
    'multiclass-mcc': <class 'chemprop.nn.metrics.MulticlassMCCMetric'>,
    'sid': <class 'chemprop.nn.metrics.SIDMetric'>,
    'wasserstein': <class '

In [20]:
metric_list = [
    nn.metrics.RMSEMetric(),
    nn.metrics.MAEMetric(),
]  # Only the first metric is used for training and early stopping

## Constructs MPNN

In [21]:
mpnn = models.MPNN(mp, agg, ffn, batch_norm, metric_list)

mpnn

MPNN(
  (message_passing): BondMessagePassing(
    (W_i): Linear(in_features=86, out_features=300, bias=False)
    (W_h): Linear(in_features=300, out_features=300, bias=False)
    (W_o): Linear(in_features=372, out_features=300, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
    (tau): ReLU()
    (V_d_transform): Identity()
    (graph_transform): Identity()
  )
  (agg): MeanAggregation()
  (bn): Identity()
  (predictor): RegressionFFN(
    (ffn): MLP(
      (0): Sequential(
        (0): Linear(in_features=300, out_features=300, bias=True)
      )
      (1): Sequential(
        (0): ReLU()
        (1): Dropout(p=0.0, inplace=False)
        (2): Linear(in_features=300, out_features=1, bias=True)
      )
    )
    (criterion): MSELoss(task_weights=[[1.0]])
    (output_transform): UnscaleTransform()
  )
  (X_d_transform): Identity()
)

# Set up trainer

In [22]:
model_output_dir = Path(f"chemprop_training/{NOW}")
monitor_mode = "min" if mpnn.metrics[0].minimize else "max"
checkpointing = ModelCheckpoint(
    model_output_dir / "checkpoints",
    "best-{epoch}-{val_loss:.2f}",
    "val_loss",
    mode=monitor_mode,
    save_last=True,
)

In [23]:
trainer = pl.Trainer(
    logger=False,
    enable_checkpointing=True,  # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
    enable_progress_bar=True,
    accelerator="auto",
    callbacks=[checkpointing],
    devices=1,
    max_epochs=50,  # number of epochs to train for
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


# Start training

In [24]:
trainer.fit(mpnn, train_loader, val_loader)

Loading `train_dataloader` to estimate number of stepping batches.
/home/gridsan/sli/.conda/envs/chemprop_v2/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=95` in the `DataLoader` to improve performance.

  | Name            | Type               | Params
-------------------------------------------------------
0 | message_passing | BondMessagePassing | 227 K 
1 | agg             | MeanAggregation    | 0     
2 | bn              | Identity           | 0     
3 | predictor       | RegressionFFN      | 90.6 K
4 | X_d_transform   | Identity           | 0     
-------------------------------------------------------
318 K     Trainable params
0         Non-trainable params
318 K     Total params
1.273     Total estimated model params size (MB)
SLURM auto-requeueing enabled. Setting signal handlers.


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/gridsan/sli/.conda/envs/chemprop_v2/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=95` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.


# Model prediction

In [25]:
results = trainer.test(mpnn, test_loader)

SLURM auto-requeueing enabled. Setting signal handlers.
/home/gridsan/sli/.conda/envs/chemprop_v2/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=95` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
 batch_averaged_test/mae    0.8292573094367981
batch_averaged_test/rmse     0.980299174785614
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


# Save the best model

In [26]:
best_model_path = checkpointing.best_model_path
model = mpnn.__class__.load_from_checkpoint(best_model_path)
p_model = model_output_dir / "best.pt"
save_model(p_model, model)

# Predicting

# Change model input here

In [27]:
model_paths = find_models([model_output_dir])
model_paths

[PosixPath('chemprop_training/2024-07-11T13-26-22/best.pt')]

In [28]:
chemprop_dir = Path.cwd().parent
test_path = chemprop_dir / "tests" / "data" / "regression" / "mol" / "mol.csv"
df_test = pd.read_csv(test_path)
test_dset = data.MoleculeDataset(test_data, featurizer)
test_loader = data.build_dataloader(test_dset, num_workers=num_workers, shuffle=False)
df_test

,smiles,lipo
0,Cn1c(CN2CCN(CC2)c3ccc(Cl)cc3)nc4ccccc14,3.54
1,COc1cc(OC)c(cc1NC(=O)CSCC(=O)O)S(=O)(=O)N2C(C)...,-1.18
2,COC(=O)[C@@H](N1CCc2sccc2C1)c3ccccc3Cl,3.69
3,OC[C@H](O)CN1C(=O)C(Cc2ccccc12)NC(=O)c3cc4cc(C...,3.37
4,Cc1cccc(C[C@H](NC(=O)c2cc(nn2C)C(C)(C)C)C(=O)N...,3.10
...,...,...
95,CC(C)N(CCCNC(=O)Nc1ccc(cc1)C(C)(C)C)C[C@H]2O[C...,2.20
96,CCN(CC)CCCCNc1ncc2CN(C(=O)N(Cc3cccc(NC(=O)C=C)...,2.04
97,CCSc1c(Cc2ccccc2C(F)(F)F)sc3N(CC(C)C)C(=O)N(C)...,4.49
98,COc1ccc(Cc2c(N)n[nH]c2N)cc1,0.20


In [29]:
# use the validation set from the training as the calibration set as an example
cal_dset = data.MoleculeDataset(val_data, featurizer)
cal_loader = data.build_dataloader(cal_dset, num_workers=num_workers, shuffle=False)

# Uncertainty predictor

In [30]:
print(uncertainty.UncertaintyPredictorRegistry)

ClassRegistry {
    'nouncertaintypredictor': <class 'chemprop.uncertainty.predictor.NoUncertaintyPredictor'>,
    'mve': <class 'chemprop.uncertainty.predictor.MVEPredictor'>,
    'ensemble': <class 'chemprop.uncertainty.predictor.EnsemblePredictor'>,
    'classification': <class 'chemprop.uncertainty.predictor.ClassPredictor'>,
    'evidential-total': <class 'chemprop.uncertainty.predictor.EvidentialTotalPredictor'>,
    'evidential-epistemic': <class 'chemprop.uncertainty.predictor.EvidentialEpistemicPredictor'>,
    'evidential-aleatoric': <class 'chemprop.uncertainty.predictor.EvidentialAleatoricPredictor'>,
    'dropout': <class 'chemprop.uncertainty.predictor.DropoutPredictor'>,
    'spectra-roundrobin': <class 'chemprop.uncertainty.predictor.RoundRobinSpectraPredictor'>,
    'dirichlet': <class 'chemprop.uncertainty.predictor.DirichletPredictor'>,
    'conformal-quantile-regression': <class 'chemprop.uncertainty.predictor.ConformalQuantileRegressionPredictor'>,
    'conformal-r

In [31]:
unc_predictor_cls = uncertainty.predictor.NoUncertaintyPredictor

# Uncertainty calibrator

In [32]:
print(uncertainty.UncertaintyCalibratorRegistry)

ClassRegistry {
    'zscaling': <class 'chemprop.uncertainty.calibrator.ZScalingCalibrator'>,
    'tscaling': <class 'chemprop.uncertainty.calibrator.TScalingCalibrator'>,
    'zelikman-interval': <class 'chemprop.uncertainty.calibrator.ZelikmanCalibrator'>,
    'mve-weighting': <class 'chemprop.uncertainty.calibrator.MVEWeightingCalibrator'>,
    'platt': <class 'chemprop.uncertainty.calibrator.PlattCalibrator'>,
    'conformal-multilabel': <class 'chemprop.uncertainty.calibrator.ConformalMultilabelCalibrator'>,
    'conformal-multiclass': <class 'chemprop.uncertainty.calibrator.ConformalMulticlassCalibrator'>,
    'conformal-adaptive': <class 'chemprop.uncertainty.calibrator.ConformalAdaptiveMulticlassCalibrator'>,
    'conformal-regression': <class 'chemprop.uncertainty.calibrator.ConformalRegressionCalibrator'>,
    'conformal-quantile-regression': <class 'chemprop.uncertainty.calibrator.ConformalQuantileRegressionCalibrator'>,
    'isotonic': <class 'chemprop.uncertainty.calibrato

In [33]:
unc_calibrator_cls = uncertainty.calibrator.ZScalingCalibrator

# Uncertainty evaluator

In [34]:
print(uncertainty.UncertaintyEvaluatorRegistry)

ClassRegistry {
    'nll-regression': <class 'chemprop.uncertainty.evaluator.NLLRegressionEvaluator'>,
    'nll-classification': <class 'chemprop.uncertainty.evaluator.NLLClassEvaluator'>,
    'nll-multiclass': <class 'chemprop.uncertainty.evaluator.NLLMultiEvaluator'>,
    'miscalibration_area': <class 'chemprop.uncertainty.evaluator.CalibrationAreaEvaluator'>,
    'ence': <class 'chemprop.uncertainty.evaluator.ExpectedNormalizedErrorEvaluator'>,
    'spearman': <class 'chemprop.uncertainty.evaluator.SpearmanEvaluator'>,
    'conformal-coverage-regression': <class 'chemprop.uncertainty.evaluator.ConformalRegressionEvaluator'>,
    'conformal-coverage-multiclass': <class 'chemprop.uncertainty.evaluator.ConformalMulticlassEvaluator'>,
    'conformal-coverage-classification': <class 'chemprop.uncertainty.evaluator.ConformalMultilabelEvaluator'>
}


In [35]:
evaluators = [
    uncertainty.evaluator.CalibrationAreaEvaluator,
    uncertainty.evaluator.SpearmanEvaluator,
]

## Load model

In [36]:
models = [load_model(model_path, multicomponent=False) for model_path in model_paths]

# Setup trainer

In [37]:
trainer = pl.Trainer(logger=False, enable_progress_bar=True, accelerator="cpu", devices=1)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


# Make uncertainty predictions

In [ ]:
predss, uncss = unc_predictor_cls(test_loader, models, trainer)
preds = torch.concat(predss, 0)
uncs = torch.concat(uncss, 0)
df_test = pd.read_csv(test_path)
df_test["pred"] = preds
df_test["unc"] = uncs
df_test

# Apply uncertainty calibration

In [ ]:
if cal_loader is not None:
    cal_predss, cal_uncss = unc_predictor_cls(cal_loader, models, trainer)
    cal_preds = torch.concat(cal_predss, 0)
    cal_uncs = torch.concat(cal_uncss, 0)
    cal_targets = cal_dset.Y
    cal_mask = torch.from_numpy(np.isfinite(cal_targets))
    cal_targets = np.nan_to_num(cal_targets, nan=0.0)
    unc_calibrator_cls.calibrate(cal_preds, cal_uncs, cal_targets, cal_mask)
    preds, uncs = unc_calibrator_cls.apply_calibration(preds, uncs)
df_test["cal_unc"] = uncs
df_test

# Uncertainty evaluation

In [ ]:
targets = test_dset.Y
mask = torch.from_numpy(np.isfinite(targets))
evaluations = []
for evaluator in evaluators:
    evaluation = evaluator(preds, uncs, targets, mask)
    evaluations.append(evaluation)
    print(f"{evaluator}: evaluation")